[Reference](https://medium.com/@nilufarmohammadi1/find-the-best-route-with-openstreetmap-using-python-da70eff5b1ac)

In [1]:
!pip install pyrosm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached cykhash-2.0.0-cp37-cp37m-linux_x86_64.whl
     |████████████████████████████████| 1.6 MB 47.9 MB/s 
  Using cached pyrobuf-0.9.3-cp37-cp37m-linux_x86_64.whl
     |████████████████████████████████| 1.0 MB 52.7 MB/s 
     |████████████████████████████████| 2.1 MB 55.6 MB/s 
     |████████████████████████████████| 16.7 MB 482 kB/s 
     |████████████████████████████████| 6.3 MB 58.2 MB/s 
  Created wheel for pyrosm: filename=pyrosm-0.6.1-cp37-cp37m-linux_x86_64.whl size=5430297 sha256=3701eddf48c678829bcbe22479e8a60b5d256dc55179f651c9c924a0a58bc151
  Stored in directory: /root/.cache/pip/wheels/e8/9e/d8/e424ffc5eaa71ccb122bb79d92ef75802c49b9b0a9366d6da4
Successfully built pyrosm


In [2]:
!pip install mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 5.2 MB/s 


In [4]:
pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 260 kB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 1.0 MB 41.3 MB/s 
     |████████████████████████████████| 11.2 MB 41.9 MB/s 
     |████████████████████████████████| 930 kB 59.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascienc

In [5]:
from pyrosm import OSM, get_data
import geopandas as gpd
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt
import time
import networkx as nx
import igraph as ig
import osmnx as ox
import folium
import json

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
osm = OSM(get_data("Southern California"))

graph_type = {}

n_drive,  e_drive = osm.get_network(nodes=True, network_type="driving")
n_cycling, e_cycling   = osm.get_network(nodes=True, network_type="cycling")
n_walk,   e_walk       = osm.get_network(nodes=True, network_type="walking")
n_service, e_service   = osm.get_network(nodes=True, network_type="driving+service")

graph_type['drive']     = ox.add_edge_travel_times(ox.add_edge_speeds(osm.to_graph(n_drive, e_drive,  graph_type="networkx")), extra_kwargs={"hv":{"car":120}})
graph_type['walk']   = ox.add_edge_travel_times(ox.add_edge_speeds(osm.to_graph(n_walk, e_walk, graph_type="networkx")))
graph_type['cycle']   = ox.add_edge_travel_times(ox.add_edge_speeds(osm.to_graph(n_cycling, e_cycling, graph_type="networkx")))
graph_type['service']   = ox.add_edge_travel_times(ox.add_edge_speeds(osm.to_graph(n_service, e_service, graph_type="networkx")))



def get_route(source_geo, dest_geo, go_type='drive', weight='travel_time',plot=True):
    source_node = ox.get_nearest_node(graph_type[go_type], source_geo)
    target_node = ox.get_nearest_node(graph_type[go_type], dest_geo)

    route = nx.shortest_path(graph_type[go_type], source_node, target_node, weight=weight)
    
    edge_lengths = ox.utils_graph.get_route_edge_attributes(graph_type[go_type], route, 'length') 
    edge_travel_time = ox.utils_graph.get_route_edge_attributes( graph_type[go_type], route, 'travel_time') 
    total_route_length = round(sum(edge_lengths), 1)
    route_travel_time  = round(sum(edge_travel_time)/60, 2)
    if plot:
      ox.plot_graph_route(graph_type[go_type], route, node_size=0, figsize=(40,40))
    return route, total_route_length, route_travel_time

Downloaded Protobuf data 'socal-latest.osm.pbf' (494.62 MB) to:
'/tmp/pyrosm/socal-latest.osm.pbf'


In [ ]:
hospitals = [
    {
       "id":"1",
       "name":"Southern California Hospital at Hollywood",
       "lat":"34.1118016",
       "long":"-118.2848717",
       "score":68,
       "empty_bed":10,
       "specialities":[
           "eye",
           "ear"
       ]
    },
    {
       "id":"2",
       "name":"Keck Hospital of USC",
       "lat":"35.7131699", 
       "long":"51.3784919",
       "score":30,
       "empty_bed":20,
       "specialities":[
           "eye",
           "ear"
        ]
    },
    {
       "id":"3",
       "name":"Southern California Hospital at Culver City",
       "lat":"35.7131699", 
       "long":"51.3784919",
       "score":32,
       "empty_bed":30,
       "specialities":[
           "eye",
           "ear"
        ]
    },
    {
       "id":"4",
       "name":"Dignity Health - California Hospital Medical Center",
       "lat":"36.38220428075101", 
       "long":"-119.01566748685103",
       "score":53,
       "empty_bed":66,
       "specialities":[
           "eye",
           "ear"
        ]
    },
    {
       "id":"5",
       "name":"LAC+USC Medical Center",
       "lat":"33.93997997052091", 
       "long":"-118.63514061133456",
       "score":43,
       "empty_bed":33,
       "specialities":[
            "eye",
            "ear"
        ]
    }
]

In [ ]:
if __name__ == "main":
#   hospitals = json.loads("hospitals.json") # read from api hospitals list
  source_loc = (34.018255, -118.313290) # just for test you must get it from gps
  nearest_hospitals = []
  for hospital in hospitals:
    hospital_loc = (hospital['lat'], hospital['long'])
    hospital_score = hospital['score'] # this is score of hospital in api just for rating
    route, route_lenght, route_time = get_route(source_loc, hospital_loc)
    # route_lenght -> distance between us and hospital in meter
    # route_time   -> duration of startpoint to hospital in minutes
    # route        -> array of point by point to hospital 
    hospital_score -= (route_time * 5) # dec 5 score per minute to arrive at the hospital
    nearest_hospitals.append({'id':hospital['id'],'score': hospital_score})
  newlist = sorted(nearest_hospitals, key=lambda d: d['score'], reverse=True) 
  print(newlist)
  # new list is most populer and nearst hospital from start point